# 手写数字识别验证

## 导入训练好的模型：

In [1]:
import torch
from torch import nn
from torchvision import transforms
import torch.nn.functional as F


import cv2
import numpy as np
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [85]:
MEAN = 0.1307
STANDARD_DEVIATION = 0.3081
class lenet(nn.Module):
    def __init__(self):
        super().__init__()
        self.dnn=torch.nn.Sequential(nn.Conv2d(1, 6, kernel_size=5,padding=2),
                          nn.Sigmoid(),
                          nn.AvgPool2d(kernel_size=2, stride=2),
                          nn.Conv2d(6, 16, kernel_size=5), nn.Sigmoid(),
                          nn.AvgPool2d(kernel_size=2, stride=2), nn.Flatten(),
                          nn.Linear(16 * 5 * 5, 120), nn.Sigmoid(),
                          nn.Linear(120, 84), nn.Sigmoid(), nn.Linear(84, 10))
    def forward(self,x):
        x = x.reshape(500, 500, 4)
        x = torch.narrow(x, dim=2, start=0, length=1)
        x = x.reshape(1, 1, 500, 500)
        
        x = F.max_pool2d(x, 14, stride=18)

        out=self.dnn(x)
        out= F.softmax(out, dim=1)
        return out



In [11]:
img=cv2.imread('../data/number/11.png',0)
for i in range(500):
    for j in range(500):
        img[i][j]=255-img[i][j]
cv2.imwrite('../data/number/reverse11.png',img)
img=img.reshape(1,1,500,500)

False

In [12]:
img

array([[[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]]], dtype=uint8)

In [13]:
img=torch.from_numpy(img).float()

In [14]:
img=F.avg_pool2d(img,14,stride=18)
#m=torch.nn.AdaptiveMaxPool2d(28)
#img=m(img)

In [15]:
img=np.array(img)

In [16]:
img=img.reshape(28,28)
cv2.imwrite('../data/number/avgpool11.png',img)

True

In [86]:
le=lenet()
le.load_state_dict(torch.load('../model_save/netmax_try4onnx.pth'))
# le=le.to(device)

<All keys matched successfully>

In [71]:
dummy_input = torch.randn(500,500,4)
le.eval()

lenet(
  (dnn): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): Sigmoid()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Sigmoid()
    (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=400, out_features=120, bias=True)
    (8): Sigmoid()
    (9): Linear(in_features=120, out_features=84, bias=True)
    (10): Sigmoid()
    (11): Linear(in_features=84, out_features=10, bias=True)
  )
)

In [72]:
dummy_input = torch.zeros(500 * 500 * 4)
torch.onnx.export(le, dummy_input, 'onnx_model_softmax500_bi_adpmaxpool.onnx', verbose=True)

RuntimeError: Unsupported: ONNX export of operator adaptive_max_pool2d, since output size is not factor of input size. Please feel free to request support or submit a pull request on PyTorch GitHub.

In [40]:
input_names = ["input"]
output_names = ["output"]
batch_size = 1
input_shape = (batch_size, 1, 28, 28)
output_shape = (batch_size, 10)
 
# 将 PyTorch 模型转换为 ONNX 格式
torch.onnx.export(
    le,  # 要转换的 PyTorch 模型
    torch.randn(input_shape),  # 模型输入的随机张量
    "lenet2.onnx",  # 保存的 ONNX 模型的文件名
    input_names=input_names,  # 输入张量的名称
    output_names=output_names,  # 输出张量的名称
    dynamic_axes={input_names[0]: {0: "batch_size"}, output_names[0]: {0: "batch_size"}}  # 动态轴，即输入和输出张量可以具有不同的批次大小
)


In [43]:
onnx_model = onnx.load("lenet2.onnx")
onnx_model_graph = onnx_model.graph
onnx_session = onnxruntime.InferenceSession(onnx_model.SerializeToString())
 
# 使用随机张量测试 ONNX 模型
x = torch.randn((1,1,28,28)).numpy()
onnx_output = onnx_session.run(output_names, {input_names[0]: x})[0]
 
print(f"PyTorch output: {le(torch.from_numpy(x)).detach().numpy()}")
print(f"ONNX output: {onnx_output}")


PyTorch output: [[ 2.2400398  -2.1915765   2.9717813  -1.7777514  -6.782894    0.37631506
  -0.42980683 -2.684063   -2.1112046  -6.4059377 ]]
ONNX output: [[ 2.2400405  -2.1915767   2.9717815  -1.777752   -6.7828927   0.37631458
  -0.42980716 -2.684062   -2.1112056  -6.4059377 ]]


In [6]:
torch.onnx.export(le,dummy_input.to(device),'../model_save/lenet.onnx',export_params=True)

In [7]:
import onnx

import onnxruntime

In [4]:

onnx_model=onnx.load("../model_save/lenet.onnx")

In [28]:

import onnxruntime as ort
 
# 加载 ONNX 模型
ort_session = ort.InferenceSession("lenet2.onnx")
 
# 准备输入信息
input_info = ort_session.get_inputs()[0]
input_name = input_info.name
input_shape = input_info.shape
input_type = input_info.type
 
 
# 运行ONNX模型
outputs = ort_session.run(input_name, dummy_input)
 
# 获取输出信息
output_info = ort_session.get_outputs()[0]
output_name = output_info.name
output_shape = output_info.shape
output_data = outputs[0]
 
print("outputs:", outputs)
print("output_info :", output_info )
print("output_name :", output_name )
print("output_shape :", output_shape )
print("output_data :", output_data )



TypeError: run(): incompatible function arguments. The following argument types are supported:
    1. (self: onnxruntime.capi.onnxruntime_pybind11_state.InferenceSession, arg0: List[str], arg1: Dict[str, object], arg2: onnxruntime.capi.onnxruntime_pybind11_state.RunOptions) -> List[object]

Invoked with: <onnxruntime.capi.onnxruntime_pybind11_state.InferenceSession object at 0x0000027670CA3AE8>, 'input', tensor([[[[-4.0397e-01, -1.3191e+00, -1.0654e+00,  2.4199e-01,  3.3594e-01,
           -6.6109e-01, -2.0142e+00,  1.6725e+00, -6.4353e-01, -1.0101e+00,
           -9.2619e-01, -1.1033e+00, -1.9078e+00, -7.1602e-01, -7.4355e-01,
            6.4440e-01,  1.1813e-01, -5.4673e-01,  7.7746e-01, -1.9279e+00,
            4.1781e-01,  2.8851e-01,  1.9069e+00, -1.3366e+00, -1.5414e+00,
           -4.2140e-01,  2.2912e-01,  4.2076e-01],
          [-9.1246e-02,  2.0045e+00, -4.1086e-01,  4.2875e-01, -1.0772e+00,
           -8.6018e-01, -9.7969e-01,  1.4211e+00, -3.2899e-01, -1.7097e+00,
         

## 处理输入图片：

In [73]:
img=cv2.imread('../data/number/10.png',0)
print("原图形状：",img.shape) 
ret,img_bi=cv2.threshold(img,90,255,cv2.THRESH_BINARY)
img_bi=255-img_bi
cv2.imwrite('../data/number/img_ori_11.png',img_bi)

原图形状： (500, 500)


True

In [74]:
img.shape[0]

500

In [75]:
x_ls=[]
y_ls=[]
img_shape=img_bi.shape
for i in range(img_shape[0]):
    for j in range(img_shape[1]):
        if img_bi[i,j] != 0:
            x_ls.append(i)
            y_ls.append(j)

In [76]:
x_min =min(x_ls)
x_max = max(x_ls)
y_min =min(y_ls)
y_max = max(y_ls)
x_d = int((x_max-x_min)/5)
y_d = int((y_max-y_min)/5)
d=max(x_d,y_d)

In [77]:
xlen=x_max-x_min
ylen=y_max-y_min
max_len=max(xlen,ylen)

In [78]:
max_len=max_len+d
max_len

438

In [79]:
x_middle=int((x_max+x_min)/2)
y_middle=int((y_max+y_min)/2)

In [80]:
y_middle

167

In [81]:
# img_bi=img_bi[x_min-x_d:x_max+x_d, y_min-y_d:y_max+y_d]
# print("二值剪切之后形状：",img_bi.shape)
# cv2.imwrite('./data/number/img_binary_0.png',img_bi)

img_bi=img_bi[max(0,x_middle-int(max_len/2)):min(x_middle+int(max_len/2),img.shape[0]), max(0,y_middle-int(max_len/2)):min(y_middle+int(max_len/2),img.shape[0])]
print("二值剪切之后形状：",img_bi.shape)
cv2.imwrite('../data/number/10_ori.png',img_bi)

二值剪切之后形状： (438, 386)


True

In [82]:
#img_bi=cv2.resize(img_bi,(28,28),interpolation=cv2.INTER_AREA)
img_bi=cv2.resize(img_bi,(28,28))
print("二值剪切变小之后形状：",img_bi.shape)
cv2.imwrite('../data/number/10_oritrans.png',img_bi)

二值剪切变小之后形状： (28, 28)


True

In [83]:
trans=transforms.ToTensor()
img_beingrate=trans(img_bi)
img_beingrate=img_beingrate.view(1,1,28,28)

## 调用模型进行识别：

In [87]:
img_input=img_beingrate
output=le(img_input)

In [88]:
output.detach().numpy()

array([[1.5943077e-07, 3.8181454e-01, 2.1393660e-03, 9.2032151e-06,
        7.6142183e-05, 4.8497744e-04, 4.3295436e-03, 4.1186082e-05,
        6.1108494e-01, 2.0063517e-05]], dtype=float32)

In [89]:
output=output.detach().numpy()

In [90]:
output

array([[1.5943077e-07, 3.8181454e-01, 2.1393660e-03, 9.2032151e-06,
        7.6142183e-05, 4.8497744e-04, 4.3295436e-03, 4.1186082e-05,
        6.1108494e-01, 2.0063517e-05]], dtype=float32)

In [ ]:
output_softmax

In [91]:
#output_softmax = output_softmax.detach().cpu().numpy()
# # 因为结果输出是二维 取第一行也就是一批量中的第一个  现在输入的一批就是一张
pred = np.argmax(output)
print(pred)

8


In [51]:
prob=[]
for i in range(10):
    prob.append(round(output_softmax[i]/sum(output_softmax),4))

NameError: name 'output_softmax' is not defined

In [ ]:
prob